# Carvana U-Net Angles+Auxiliary

## Imports

In [1]:
from keras.layers.advanced_activations import PReLU
from keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint, TensorBoard
import keras.backend as K
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from optimizers.AdamAccumulate import AdamAccumulate
from models.u_net_heng_aux import UNet_Heng_Aux
from utilities.submit import generate_submit
from utilities import utils_masks as utils
from utilities.losses import weighted_bce_dice_loss, dice_value

%load_ext autoreload
%autoreload 2
%matplotlib inline

Using TensorFlow backend.


## Preparing Data

In [3]:
utils.set_results_reproducible()
input_size = 1024
num_pseudo_data = 6000
train_path = "inputs/train_hq/{}.jpg" 
train_mask_path = "inputs/train_masks/{}_mask.gif"
train_df = pd.read_csv('inputs/train_masks.csv')
test_path = "inputs/test_hq/{}.jpg"
test_mask_path = "outputs/test_hq_masks/{}_mask.gif"
test_df = pd.read_csv('inputs/sample_submission.csv')

ids_train = train_df['img'].map(lambda s: s.split('.')[0])
ids_train_split, ids_valid_split = train_test_split(ids_train, test_size=0.2, random_state=42)

test_df = test_df.sample(n=num_pseudo_data)
ids_test_split = test_df['img'].map(lambda s: s.split('.')[0])

print('#Training = {0}+{1}   #Validation = {2}'
      .format(len(ids_train_split), len(ids_test_split), len(ids_valid_split)))

bbox_file_path = 'inputs/data_bbox.csv'
bboxes = utils.get_bboxes(bbox_file_path)

def get_pseudo_ids(batch_size, accum_iters):
    return utils.make_list_ids(ids_train_split, ids_test_split, batch_size, accum_iters)

def valid_generator(batch_size, outputs=None):
    return utils.valid_generator(train_path, train_mask_path, ids_valid_split,
                                 input_size, batch_size, bboxes, outputs=outputs)

def pseudo_generator(batch_size, pseudo_ids, outputs=None):
    return utils.pseudo_generator(train_path, train_mask_path, test_path, test_mask_path, pseudo_ids,
                                  input_size, batch_size, bboxes, outputs=outputs,
                                  augmentations=['HUE_SATURATION', 'SHIFT_SCALE', 'FLIP'])

#Training = 4070+6000   #Validation = 1018


## Create Model

## Fit Models

In [4]:
epochs = 150
batch_size = 1
accum_iters = 32
auxiliaries = [False, False, False, False, True, True]
outputs = {'aux_out4':2**-4, 'aux_out5':2**-5, 'main_out':1}
weights = {'aux_out4':0.2, 'aux_out5':0.05, 'main_out':1.}

model = UNet_Heng_Aux((input_size, input_size, 3), auxiliaries=auxiliaries)
model.compile(optimizer=AdamAccumulate(accum_iters=accum_iters), 
              loss=weighted_bce_dice_loss, metrics=[dice_value], loss_weights=weights)
run_name = utils.get_run_name('weights/final/{}.hdf5', 'unet-ultimate')
weights_path = 'weights/final/{}.hdf5'.format(run_name)

callbacks = [EarlyStopping(monitor='val_main_out_dice_value',
                           patience=3,
                           verbose=1,
                           min_delta=1e-4,
                           mode='max'),
             ReduceLROnPlateau(monitor='val_main_out_dice_value',
                               factor=0.1,
                               patience=2,
                               verbose=1,
                               epsilon=1e-4,
                               mode='max'),
             ModelCheckpoint(monitor='val_main_out_dice_value',
                             filepath=weights_path,
                             save_best_only=True,
                             save_weights_only=True,
                             mode='max'),
             TensorBoard(log_dir='logs/{}'.format(run_name), batch_size=batch_size)]

model.load_weights('weights/unet-heng-aux-2017-09-25-1603.hdf5')
K.set_value(model.optimizer.lr, 1e-4)

pseudo_ids = get_pseudo_ids(batch_size, accum_iters)
print('Starting run "{}"'.format(run_name))
model.fit_generator(generator=pseudo_generator(batch_size, pseudo_ids, outputs), 
                    steps_per_epoch=np.ceil(float(len(pseudo_ids)) / float(batch_size)),
                    epochs=epochs,
                    verbose=1,
                    callbacks=callbacks,
                    validation_data=valid_generator(batch_size, outputs),
                    validation_steps=np.ceil(float(len(ids_valid_split)) / float(batch_size)))

Starting run "unet-ultimate-2017-09-26-2148"
Epoch 1/150
24032/24032 [==============================] - 13383s - loss: 0.5989 - aux_out4_loss: 0.4625 - aux_out5_loss: 0.4687 - main_out_loss: 0.4830 - aux_out4_dice_value: 0.9952 - aux_out5_dice_value: 0.9930 - main_out_dice_value: 0.9969 - val_loss: 0.5961 - val_aux_out4_loss: 0.4574 - val_aux_out5_loss: 0.4639 - val_main_out_loss: 0.4815 - val_aux_out4_dice_value: 0.9954 - val_aux_out5_dice_value: 0.9938 - val_main_out_dice_value: 0.9968
Epoch 2/150
24032/24032 [==============================] - 13379s - loss: 0.5985 - aux_out4_loss: 0.4617 - aux_out5_loss: 0.4670 - main_out_loss: 0.4828 - aux_out4_dice_value: 0.9955 - aux_out5_dice_value: 0.9940 - main_out_dice_value: 0.9970 - val_loss: 0.5959 - val_aux_out4_loss: 0.4571 - val_aux_out5_loss: 0.4630 - val_main_out_loss: 0.4813 - val_aux_out4_dice_value: 0.9955 - val_aux_out5_dice_value: 0.9943 - val_main_out_dice_value: 0.9968
Epoch 3/150
24032/24032 [==============================] - 

KeyboardInterrupt: 

## Mask Prediction

In [ ]:
from PIL import Image
import cv2
from glob import glob

In [ ]:
model.load_weights('weights/unet-2017-09-02-1809.hdf5')

In [ ]:
imgs_names = glob('inputs/train/*.jpg')

In [ ]:
num = 1
input_size = 1024
pred_mask_path = "outputs/pred_train_masks/{}_mask.gif"
for i in range(len(imgs_names)):
    im = cv2.imread(imgs_names[i])
    im = cv2.resize(im, (input_size, input_size), interpolation=cv2.INTER_LINEAR)
    im = np.expand_dims(im, axis=0)
    im = np.array(im, np.float32) / 255
    #print im.shape
    pred_mask = model.predict(im, batch_size=1, verbose=2)
    #print pred_mask.shape
    pred_mask = np.squeeze(pred_mask, axis=[0, 3])
    #print pred_mask.shape
    pred_mask = cv2.resize(pred_mask, (1918, 1280), interpolation=cv2.INTER_LINEAR)
    #print pred_mask.shape
    pred_mask = (pred_mask > 0.95)
    pred_mask = Image.fromarray((pred_mask * 255).astype(np.uint8), mode='L')
    im_name = imgs_names[i].split('/')[-1]
    im_id = im_name.split('.')[0]
    pred_mask.save(pred_mask_path.format(im_id))
    
    if num%500==0:
        print '{0}/{1}'.format(num, len(imgs_names))
    num += 1

In [ ]:
#val_imgs, _ = next(valid_generator(len(ids_valid_split)))
train_imgs, _ = next(train_generator(len(ids_train_split[4000:])))

In [ ]:
#print val_imgs.shape
print train_imgs.shape

In [ ]:
#val_pred_masks = model.predict(val_imgs, batch_size=16, verbose=1)
train_pred_masks = model.predict(train_imgs, batch_size=16, verbose=1)

In [ ]:
preds = np.squeeze(train_pred_masks, axis=3)
np.shape(preds)

In [ ]:
pred_mask_path = "outputs/valid_masks/{}_mask.gif"

In [ ]:
for i in range(train_imgs.shape[0]):
    mask = preds[i]
    mask = cv2.resize(mask, (1918, 1280), interpolation=cv2.INTER_LINEAR)
    mask = (mask > 0.95)
    mask = Image.fromarray((mask * 255).astype(np.uint8), mode='L')
    mask.save(pred_mask_path.format(ids_valid_split.values[i]))

## Validation

In [ ]:
def np_dice_value(y_true, y_pred):
    smooth = 1.
    y_true_f = y_true.flatten()
    y_pred_f = y_pred.flatten()
    intersection = np.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (np.sum(y_true_f) + np.sum(y_pred_f) + smooth)

### Prediction

In [ ]:
run_name = 'unet-2017-09-03-1739'
model.load_weights('weights/{}.hdf5'.format(run_name))

val_imgs, val_masks = next(valid_generator(len(ids_valid_split)))
val_imgs = np.array(val_imgs)
val_masks = np.array(val_masks)
val_pred_masks = model.predict(val_imgs, batch_size=1)
masks_val_dices = [np_dice_value(mask, pred_mask) for (mask, pred_mask) in zip(val_masks, val_pred_masks)]

### Display the worst predicted mask for validation examples

In [ ]:
index = np.argsort(masks_val_dices)[7]
id = ids_valid_split.values[index]
utils.show_mask(train_path.format(id), val_masks[index].squeeze(), val_pred_masks[index].squeeze(), show_img=False)
print id, masks_val_dices[index]

In [ ]:
indices = np.argsort(masks_val_dices[masks_val_dices <= 99.6])
for id in indices:
        print(masks_val_dices[id])

### Histogram

In [ ]:
hist, bins = np.histogram(masks_val_dices, bins=50)
width = 0.7 * (bins[1] - bins[0])
center = (bins[:-1] + bins[1:]) / 2
plt.bar(center, hist, align='center', width=width)
plt.show()

### Visualization

In [ ]:
indices = np.random.randint(len(ids_valid_split), size=3)
for index in indices:
    id = ids_valid_split.values[index]
    utils.show_mask(train_path.format(id), val_masks[index].squeeze(), val_pred_masks[index].squeeze(),
                    show_img=True, bbox = bboxes[id])

## Test

### Load Model

In [ ]:
# Create model first if required
run_name = 'unet-2017-08-20-5'
model.load_weights('weights/{}.hdf5'.format(run_name))

### Generate Submit

In [ ]:
batch_size = 16
threshold = 0.5
test_path = 'inputs/test1/' #'inputs/test/'
test_masks_path = 'outputs/test1_masks/' #None
generate_submit(model, input_size, batch_size, threshold, test_path, 'outputs/', run_name, test_masks_path)

### Visualization

In [ ]:
utils.show_test_masks(test_path, test_masks_path)